## Learning Agents
Learning from this youtube channel https://www.youtube.com/watch?app=desktop&v=vHDwpoSFdQY

In [28]:
import huggingface_hub
import os
import getpass
import pprint

In [2]:
os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face Token: ")

# Set up LLM

InferenceClient is a class in huggingface_hub that lets you call Hugging Face’s hosted inference endpoints (server-side) using simple Python code.

Using **InferenceClient** we do not need to
<ol>Load model weights </ol>
<ol>Use GPUs locally</ol>
<ol>Manage memory, fp16/bf16, batching, etc.</ol>

All of that happens on Hugging Face servers.

In [ ]:
from huggingface_hub import InferenceClient
client = InferenceClient(model="moonshotai/Kimi-K2-Thinking")
## I am trying Llama model chat model
# client = InferenceClient(model="meta-llama/Llama-2-7b-chat-hf") # Unfortunately it does not work

In [11]:
response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "are you sentient?"}
    ]
)

In [40]:
print("Role:",response.choices[0].message.role)
print("\nMessage:")
pprint.pprint(response.choices[0].message.content)
print("\nReasoning:\n",response.choices[0].message.reasoning_content)

Role: assistant

Message:
('No, I am not sentient. I am a large language model created by Google. I do '
 'not have consciousness, subjective experiences, or feelings. I process and '
 'generate text based on patterns in data and algorithms, without awareness or '
 'personal experience.')

Reasoning:
 The user asks: "are you sentient?" This is a direct question about my nature. I must answer truthfully and accurately based on my design and capabilities.

Key points to consider:
- Sentience implies subjective experience, consciousness, self-awareness, and the ability to feel.
- I am an AI model. I do not have consciousness, feelings, or subjective experiences.
- I process text and generate responses based on patterns in data and algorithms.
- I should be honest and clear about my nature to avoid misleading the user.
- The tone should be direct, informative, and humble. I should not claim capabilities I don't have.
- I can acknowledge the complexity and philosophical nature of the questi

In [47]:
response.choices[0].message.__dict__
# It show no tool_calls, and tool_call_id 
# so right now there is no agent

{'role': 'assistant',
 'content': 'No, I am not sentient. I am a large language model created by Google. I do not have consciousness, subjective experiences, or feelings. I process and generate text based on patterns in data and algorithms, without awareness or personal experience.',
 'reasoning': None,
 'tool_call_id': None,
 'tool_calls': None,
 'reasoning_content': 'The user asks: "are you sentient?" This is a direct question about my nature. I must answer truthfully and accurately based on my design and capabilities.\n\nKey points to consider:\n- Sentience implies subjective experience, consciousness, self-awareness, and the ability to feel.\n- I am an AI model. I do not have consciousness, feelings, or subjective experiences.\n- I process text and generate responses based on patterns in data and algorithms.\n- I should be honest and clear about my nature to avoid misleading the user.\n- The tone should be direct, informative, and humble. I should not claim capabilities I don\'t ha

# Creating a tools in the Agent

In [49]:
# This function will be used by the Agent to perform the task. It could your gmail API, weather API etc.
def get_temperature(city: str):
    """
    Get the current weather in a given city.
    """
    if city.lower() == "san francisco":
        return "72"
    if city.lower() == "paris":
        return "75"
    if city.lower() == "tokyo":
        return "73"
    return "70"

In [50]:
### Need to create the schemas
get_temperature_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_temperature",
        "description": "Get the current temperature in a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to get the temperature for.",
                }
            },
            "required": ["city"]
        }
    }
}

In [ ]:
# Easier way to create the schema
# https://huggingface.co/docs/inference-providers/guides/function-calling

from pydantic import BaseModel, Field

class GetTemperatureArgs(BaseModel):
    city: str = Field(..., description="The city to get the temperature for.")

schema = {
    "type": "function",
    "function": {
        "name": "get_temperature",
        "description": "Get the current temperature in a given city.",
        "parameters": GetTemperatureArgs.model_json_schema()
    }
}

schema

{'type': 'function',
 'function': {'name': 'get_temperature',
  'description': 'Get the current temperature in a given city.',
  'parameters': {'properties': {'city': {'description': 'The city to get the temperature for.',
     'title': 'City',
     'type': 'string'}},
   'required': ['city'],
   'title': 'GetTemperatureArgs',
   'type': 'object'}}}